# Problema 2:
Determine la deformación y los esfuerzos principales en función de la posición para una chapa con
un agujero en el centro sujeta a tracción por sus extremos, como se muestra en la figura. Las
dimensiones de la chapa son de 20 x 10 pulgadas y su espesor de una pulgada. El radio del agujero
central es también de 1 pulgada. Tome E = 30 x 106
 psi y n = 0.30.
Analice el problema con número creciente de elementos y estudie su convergencia

Aprovechando la simetría del ejercicio, se simula un cuarto de la pieza:

                                           l2
                                  p4------------------p3
                                   l                   l
                               l1  l           S       l
            Empotrado              l                   l    l3     Traccionado
                                   p5-----\            l
                                   l      l            l
                                   p1----p6------------p2 
                                               l4


In [1]:
import gmsh
import numpy as np
import copy

GL = 2     # Grados de libertad
E = 30e6   # psi Modulo de elasticidad
v = 0.3    # Módulo de Poisson
T = 1000   # Tensión de tracción

In [2]:
gmsh.initialize()
gmsh.model.add('ej2')

Definición de puntos

In [3]:
# Sección rectangular

lc = 4                                       # Refinamiento del mallado
H = 5                                        # Alto en pulgadas
L = 10                                       # Largo en pulgadas
Es = 1                                       # Espesor en pulgadas
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc)
p2 = gmsh.model.geo.addPoint(L, 0, 0, lc) 
p3 = gmsh.model.geo.addPoint(L, H, 0, lc) 
p4 = gmsh.model.geo.addPoint(0, H, 0, lc) 

# Sección circular central (Agujero)

r = 1                                        # Radio en pulgadas
la = 0.1                                     # Refinamiento del mallado cerca del círculo
p5 = gmsh.model.geo.addPoint(0, r, 0, la)
p6 = gmsh.model.geo.addPoint(r, 0, 0, la)

Definición de lineas entre los puntos y las uno para formar las curvas

In [4]:
# Sección llena

l1 = gmsh.model.geo.addLine(p5, p4)
l2 = gmsh.model.geo.addLine(p4, p3)
l3 = gmsh.model.geo.addLine(p3, p2)
l4 = gmsh.model.geo.addLine(p2, p6)

Rect = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])

# Semicírculo en una esquina

semi = gmsh.model.geo.addCircleArc(p5,p1,p6)

Circ = gmsh.model.geo.addCurveLoop([semi])

* Determino la superficie rectancular descontando el círculo
* Sincronizo los elementos geométricos

In [5]:
S = gmsh.model.geo.addPlaneSurface([Rect, Circ])

gmsh.model.geo.synchronize()

Condiciones de contorno: defino los grupos físicos

In [6]:
# Empotramientos (cortes de simetría)

EmpX = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1,EmpX, "EmpX")

EmpY = gmsh.model.addPhysicalGroup(1, [l1])
gmsh.model.setPhysicalName(1,EmpY, "EmpY")

In [7]:
# Contornos traccionados

Trac = gmsh.model.addPhysicalGroup(1, [l3])
gmsh.model.setPhysicalName(1,Trac,'Trac')

In [8]:
# Superficie 

Sup = gmsh.model.addPhysicalGroup(2,[S])
gmsh.model.setPhysicalName(2, Sup, 'Sup')

In [9]:
# Eliminación del punto 1 auxiliar

trash = gmsh.model.addPhysicalGroup(0, [p1])
gmsh.model.setPhysicalName(0,trash, "trash")

Generación del mallado indicando su dimensión

In [10]:
gmsh.model.geo.synchronize()

In [11]:
gmsh.model.mesh.generate(2)   # hay que correrlo dos veces, sino no funciona, pq?

Extracción de datos de mallado generado

In [12]:
# Nodos

nodos = gmsh.model.mesh.get_nodes()      # Información de los nodos del mallado
#nodos[1].shape
N = nodos[0].shape[0]                    # Número de nodos
MN = nodos[1].reshape(N , 3)             # Matriz de nodos

# Elementos

e_tags, e = gmsh.model.mesh.get_elements_by_type(2)  # Información de los elementos tipo 2 (triangulares)
MC = e.reshape([e_tags.shape[0],3])      # Matriz de conectividad

# Traccionados

ent_trac = gmsh.model.getEntitiesForPhysicalGroup(1, Trac)      # Entidad de elementos traccionados
t1, t2, long_trac = gmsh.model.mesh.getElements(1, ent_trac[0]) 
Long_trac = long_trac[0].reshape(t2[0].shape[0],2)  # Nodos de cada elemento de la entidad

Calculo las fuerzas

In [13]:
F = np.zeros((2*N, 1))

for i, line in enumerate(Long_trac):
    n1 = int(line[0]-1)
    n2 = int(line[1]-1)
    long = np.abs(MN[n1,1]-MN[n2,1])
    Floc = (T * long * Es)/2 * np.array([[1],[1]])
    F[ np.array([2*n1, 2*n2]) ] += Floc


Saco datos del mallado

In [18]:
Nx_emp = gmsh.model.mesh.get_nodes_for_physical_group(1,EmpX)
Ny_emp = gmsh.model.mesh.get_nodes_for_physical_group(1,EmpY)

In [19]:
s= []
for n, nod in enumerate(Nx_emp[0]):
    s.append(2*nod)
    s.append(2*nod+1)
for n, nod in enumerate(Ny_emp[0]):
    s.append(2*nod)
    s.append(2*nod+1)

s = np.array(s).astype(int)

r = np.delete( np.arange(2*N) , s )

luego saco de r todo lo que puse en s:

In [16]:
el = len(MC)      # cantidad de elementos
K = np.zeros([N*GL, N*GL])
Esf= []
Bs=[]
for a in range(el):
    B = np.zeros([3,6])
    c = []              #coordenadas de los nodos del elemento
    for i in MC[a]:
        aux=MN[int(i-1)].copy()
        aux[2]=1
        c.append(aux)
        b=[0,0,0]
        g=[0,0,0]
    A = np.linalg.det(c)/2
    for d in range(3): 
        b[d] = c[d-2][1]- c[d-1][1]
        g[d] = c[d-1][0]- c[d-2][0]
    for j in range(3):
        B0=[[b[j],0],[0, g[j]], [g[j], b[j]]]
        B[0][j*2]=b[j]
        B[1][2*j+1]=g[j]
        B[2][j*2]=g[j]
        B[2][j*2+1]=b[j]
    B = B/(2*A)
    Bs.append(B)
    D = (E/(1-(v**2)))*np.array([[1,v,0],[v,1,0],[0,0,(1-v)/2]])
    k = Es*abs(A)*np.dot(B.transpose(), np.dot(D,B) )
    K0 = np.zeros([N*2, N*2])
    for i in range(3):
        nx=int(MC[a][i]-1)
        for j in range(3):
            ny=int(MC[a][j]-1)
            K0[np.ix_([nx*2,nx*2+1], [ny*2,ny*2+1])]+=k[i*2:i*2+2, j*2:j*2+2]
    K+=K0

In [17]:
d = np.zeros(N*GL)
f = np.zeros(N*GL)

dd = np.linalg.solve(K[np.ix_(r,r)], F[r])

for i in range(len(r)):
    d[r[i]]=dd[i]

F=K.dot(d)
B=np.array(B)
D=np.array(D)

for i in range(e):
    j=np.array(MC[i])*2 
    Esf.append( np.dot( np.dot(D,Bs[i]), d[[j[0],j[0]+1,j[1],j[1]+1,j[2],j[2]+1]] ) ) 
    


LinAlgError: Singular matrix